In [13]:
import webbrowser
import os
import requests
import bs4
from urllib.request import urlopen
import pandas as pd

In [ ]:
"""
과제 (다음주 목요일까지)
멜론, 벅스, 지니 사이트에서
웹 크롤링 하여
음악순위 탑 50 가져오기 (순위, 곡제목, 가수)
각각 엑셀로 저장하여 (총 3개 - 멜론, 벅스, 지니)
토탈 음악파일 엑셀 1개로 합치기
시각화하여 표현하기
"""

In [ ]:
melon_url = 'https://www.melon.com/chart/index.htm'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(melon_url, headers = headers)
response.encoding = 'UTF-8'
html_target = response.text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<head>\r\n\t\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n\r\n\t\r\n\r\n\t\r\n\r\n\t<title>멜론차트>TOP100>멜론</title>\r\n\t<meta name="keywords" content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" />\r\n\t<meta name="description" content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" />\r\n\t<meta name="naver-site-verification" content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" />\r\n\t<meta name="google-site-verification" content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" />\r\n\t<meta property="fb:app_id" content="4022717807957185"/>\r\n\t<meta property="og:title" content="Melon"/>\r\n\t<meta property="og:image" content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png

In [26]:
# 응답 객체의 'status_code'를 출력하여 요청이 성공했는지 확인
# "200"이면 요청이 성공적이라는 의미
print(response.status_code)

200


In [ ]:
# --- BeautifulSoup 객체 생성 및 타입 확인 ---
# 가져온 HTML 콘텐츠를 BeautifulSoup을 사용하여 파싱
bs_melon = bs4.BeautifulSoup(html_target, 'html.parser')

# BeautifulSoup 객체의 타입을 출력하여 객체가 잘 생성되었는지 확인
print(type(bs_melon))

<class 'bs4.BeautifulSoup'>


In [30]:
# 웹 스크래핑에 용이한 형태로 HTML 구조를 보기 위해 'prettify()' 메서드 사용
# 이를 통해 HTML 구조를 들여쓰기가 적용된 보기 쉬운 형태로 출력
print(bs_melon.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   멜론차트&gt;TOP100&gt;멜론
  </title>
  <meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
  <meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
  <meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
  <meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
  <meta content="4022717807957185" property="fb:app_id"/>
  <meta content="Melon" property="og:title"/>
  <meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
  <meta content="음악이 필요한 순간, 멜론" property="og:description"/>
  <meta content="http://www.melon.com/chart/index.htm" property="og:url"/>
  <meta content="website" property="og

In [ ]:
# --- CSS 선택자 사용하여 데이터 추출 ---

# rank01 클래스에 해당하는 HTML 요소들을 선택
rank01 = bs_melon.select('.wrap .wrap_song_info .ellipsis.rank01')

# 빈 리스트 'title' 생성 (곡 제목을 저장할 리스트)
title = []

# rank01에서 첫 100개의 요소에 대해 반복
for a in rank01[:100]:
    # 각 요소의 텍스트를 추출하여 공백 및 개행 문자 제거 후 'title' 리스트에 추가
    title.append(a.text.strip())
    
    # 각 곡 제목을 출력
    print(a.text)


Good Goodbye


타임캡슐


Blue Valentine


ONE MORE TIME


Golden


첫 눈


Drowning


SPAGHETTI (feat. j-hope of BTS)


멸종위기사랑


달리 표현할 수 없어요


어제보다 슬픈 오늘


뛰어(JUMP)


FAMOUS


모르시나요(PROD.로코베리)


사랑하게 될 거야


Soda Pop


시작의 아이 ❍


한번 더 이별


내게 사랑이 뭐냐고 물어본다면


어떻게 이별까지 사랑하겠어, 널 사랑하는 거지


HOME SWEET HOME (feat. 태양, 대성)


toxic till the end


너에게 닿기를


IRIS OUT


XOXZ


천상연


너의 모든 순간


운명 (2025)


Rich Man


Whiplash


나는 반딧불


Hollywood Action


청춘만화


시작의 아이


소나기


모든 날, 모든 순간 (Every day, Every Moment)


HAPPY


오늘만 I LOVE YOU


Flower


like JENNIE


그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))


body


APT.


Never Ending Story


사랑은 늘 도망가


예뻤어


한 페이지가 될 수 있게


FOCUS


가만히 눈을 감고


Die With A Smile


Welcome to the Show


나였으면


Love wins all


All I Want for Christmas Is You


청혼하지 않을 이유를 못 찾았어


Dirty Work


다정히 내 이름을 부르면


주저하는 연인들을 위해


REBEL HEART


TOO BAD (feat. Anderson .Paak)


눈이 오잖아(Feat.헤이즈)


사랑인가 봐


인사


힙합보단 사랑, 사랑보단 돈 (Feat. 베이식)


WICKED


STYLE


헤어지자 말해요


빌려온 고양이 (Do 

In [90]:
rank02 = bs_melon.select('.wrap .wrap_song_info .ellipsis.rank02 .checkEllipsis')
name = []
for a in rank02[:100] :
    name.append(a.text)
    print(a.text)
name

화사 (HWASA)
다비치
NMIXX
ALLDAY PROJECT
HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demon Hunters Cast
EXO
WOODZ
LE SSERAFIM (르세라핌), j-hope
이찬혁
로이킴
우디 (Woody)
BLACKPINK
ALLDAY PROJECT
조째즈
한로로
KPop Demon Hunters Cast, Danny Chung, Saja Boys, Andrew Choi, Neckwav, Kevin Woo, samUIL Lee
박다혜, 마크툽 (MAKTUB)
이창섭
로이킴
AKMU (악뮤)
G-DRAGON
로제 (ROSÉ)
10CM
Kenshi Yonezu
IVE (아이브)
이창섭
성시경
먼데이 키즈, 이이경
aespa
aespa
황가람
BOYNEXTDOOR
이무진
마크툽 (MAKTUB)
이클립스 (ECLIPSE)
폴킴
DAY6 (데이식스)
BOYNEXTDOOR
오반(OVAN)
제니 (JENNIE)
너드커넥션 (Nerd Connection)
다영 (DAYOUNG)
로제 (ROSÉ), Bruno Mars
아이유
임영웅
DAY6 (데이식스)
DAY6 (데이식스)
Hearts2Hearts (하츠투하츠)
DK(디셈버)
Lady Gaga, Bruno Mars
DAY6 (데이식스)
나윤권, 도경수(D.O.)
아이유
Mariah Carey
이무진
aespa
경서예지, 전건호
잔나비
IVE (아이브)
G-DRAGON
이무진
멜로망스
범진
노아주다 (noahjooda)
ALLDAY PROJECT
Hearts2Hearts (하츠투하츠)
박재정
아일릿(ILLIT)
이예은, 아샤트리, 전건호
이무진
아일릿(ILLIT)
아샤트리, 이예은, 전건호
TWS (투어스)
aespa
임재현
한로로
순순희(지환)
KPop Demon Hunters Cast, Danny Chung, Saja Boys, Andrew Choi, Neckwav, Kevin Woo, samUIL Lee
임영웅
프로미스나인
IVE (아이브)
QWER
Ne

['화사 (HWASA)',
 '다비치',
 'NMIXX',
 'ALLDAY PROJECT',
 'HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demon Hunters Cast',
 'EXO',
 'WOODZ',
 'LE SSERAFIM (르세라핌), j-hope',
 '이찬혁',
 '로이킴',
 '우디 (Woody)',
 'BLACKPINK',
 'ALLDAY PROJECT',
 '조째즈',
 '한로로',
 'KPop Demon Hunters Cast, Danny Chung, Saja Boys, Andrew Choi, Neckwav, Kevin Woo, samUIL Lee',
 '박다혜, 마크툽 (MAKTUB)',
 '이창섭',
 '로이킴',
 'AKMU (악뮤)',
 'G-DRAGON',
 '로제 (ROSÉ)',
 '10CM',
 'Kenshi Yonezu',
 'IVE (아이브)',
 '이창섭',
 '성시경',
 '먼데이 키즈, 이이경',
 'aespa',
 'aespa',
 '황가람',
 'BOYNEXTDOOR',
 '이무진',
 '마크툽 (MAKTUB)',
 '이클립스 (ECLIPSE)',
 '폴킴',
 'DAY6 (데이식스)',
 'BOYNEXTDOOR',
 '오반(OVAN)',
 '제니 (JENNIE)',
 '너드커넥션 (Nerd Connection)',
 '다영 (DAYOUNG)',
 '로제 (ROSÉ), Bruno Mars',
 '아이유',
 '임영웅',
 'DAY6 (데이식스)',
 'DAY6 (데이식스)',
 'Hearts2Hearts (하츠투하츠)',
 'DK(디셈버)',
 'Lady Gaga, Bruno Mars',
 'DAY6 (데이식스)',
 '나윤권, 도경수(D.O.)',
 '아이유',
 'Mariah Carey',
 '이무진',
 'aespa',
 '경서예지, 전건호',
 '잔나비',
 'IVE (아이브)',
 'G-DRAGON',
 '이무진',
 '멜로망스',
 '범진',
 '노아주다 (noahjoo

In [97]:
# 딕셔너리 생성
music_dict = dict(zip(title, name))

# 딕셔너리에서 데이터프레임으로 변환
df = pd.DataFrame(list(music_dict.items()), columns=['Title', 'Artist'])

# 인덱스를 1번부터 시작하도록 설정
df.reset_index(drop=True, inplace=True)  # 기존 인덱스 제거
df.index = df.index + 1  # 새로운 인덱스는 1부터 시작

# 인덱스를 컬럼으로 추가
df_reset = df.reset_index()

df_reset

,index,Title,Artist
0,1,Good Goodbye,화사 (HWASA)
1,2,타임캡슐,다비치
2,3,Blue Valentine,NMIXX
3,4,ONE MORE TIME,ALLDAY PROJECT
4,5,Golden,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo..."
...,...,...,...
95,96,"바이, 썸머",아이유
96,97,Santa Tell Me,Ariana Grande
97,98,BBUU!,PLAVE
98,99,들꽃이 될게요,임영웅
